In [ ]:
import pandas as pd

# Load your dataset
file_path = "DataSet.xlsx"
data = pd.read_excel(file_path, sheet_name="Database")

# Create the new Enrolled column:
# If Student ID has a value → Enrolled = 1, else Enrolled = 0
data["Enrolled"] = data["Student ID"].notna().astype(int)
data = data.drop(columns=["Student ID"])

# Step 1: Show population (non-null counts) of each column
print("=== Column Population (non-missing values) ===")
population = data.count()
print(population)

# Step 2: Show percentage population for better insight
print("\n=== Column Population Percentage ===")
population_percent = (data.count() / len(data)) * 100
print(population_percent)

# Step 3: Drop columns with less than 60% non-missing data
threshold = len(data) * 0.6
data_cleaned = data.dropna(axis=1, thresh=threshold)

# Step 4: Show which columns got dropped
dropped_columns = set(data.columns) - set(data_cleaned.columns)
print("\n=== Dropped Columns (less than 60% population) ===")
print(dropped_columns)

# Step 5: Verify the new dataset structure
print("\n=== Cleaned Dataset Info ===")
print(data_cleaned.info())

=== Column Population (non-missing values) ===
ID                            20438
School Year                   20438
School Term                   20438
Campus Code                   20438
Program (First Choice)        20438
Program (Second Choice)       12545
Birth Date                    20438
Birth Place                   17172
Birth City                    17170
Place of Birth (Province)     17115
Gender                        17172
Citizen of                    17828
Religion                      17169
Civil Status                  17172
Current Region                15573
Current Province              16354
City/Municipality             16354
Current Brgy.                 16350
Current Street                16010
Current Postal Code           15980
Complete Present Address      16354
Telephone No.                  2625
Mobile Number                 17523
Email                         20438
Permanent Country             16354
Permanent Region              15521
Permanent Provinc

In [ ]:
print(data_cleaned.columns.tolist())


['ID', 'School Year', 'School Term', 'Campus Code', 'Program (First Choice)', 'Program (Second Choice)', 'Birth Date', 'Birth Place', 'Birth City', 'Place of Birth (Province)', 'Gender', 'Citizen of', 'Religion', 'Civil Status', 'Current Region', 'Current Province', 'City/Municipality', 'Current Brgy.', 'Current Street', 'Current Postal Code', 'Complete Present Address', 'Mobile Number', 'Email', 'Permanent Country', 'Permanent Region', 'Permanent Province', 'Permanent City', 'Permanent Brgy.', 'Permanent Street', 'Permanent Postal Code', 'Complete Permanent Address', 'Birth Country', 'Student Type', 'Last School Attended', 'School Type']


In [ ]:


# Keep only the selected columns
filtered_data = data_cleaned

# Normalize text: lowercase, strip spaces, unify spacing
skip_cols = ["Current Postal Code", "Permanent Postal Code", "Enrolled"]
for col in filtered_data.select_dtypes(include="object").columns:
    if col not in skip_cols:
        filtered_data[col] = (
            filtered_data[col]
            .astype(str)            # ensure everything is string
            .str.lower()            # convert to lowercase
            .str.strip()            # remove leading/trailing spaces
            .str.replace(r"\s+", " ", regex=True)  # collapse multiple spaces
            .replace({"nan": None, "none": None})  # clean placeholder "nan"/"none"
        )

# Save to new file (optional)
filtered_data.to_excel("Filtered_DataSet.xlsx", index=False)

# Preview first few rows
print(filtered_data.head())


/tmp/ipython-input-2188889483.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[col] = (
/tmp/ipython-input-2188889483.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[col] = (
/tmp/ipython-input-2188889483.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

  ID School Year School Term Campus Code Program (First Choice)  \
0  4        2022           1        lipa                bsa-mla   
1  6        2022           1        lipa                bsa-mla   
2  7        2022           1        lipa               bsmt-mla   
3  8        2022           1        lipa                   bsce   
4  9        2022           1        lipa                   bsce   

                          Program (Second Choice)           Birth Date  \
0               bachelor of arts in communication  2004-01-15 00:00:00   
1  bachelor of science in business administration  2003-11-25 00:00:00   
2                  bachelor of science in nursing  2004-04-07 00:00:00   
3   bachelor of science in electrical engineering  2003-11-28 00:00:00   
4             bachelor of science in architecture  2004-05-19 00:00:00   

                        Birth Place     Birth City Place of Birth (Province)  \
0                       manila, ncr         manila                      

In [ ]:
from scipy.stats import chi2_contingency

# Load dataset
data = pd.read_excel("Filtered_DataSet.xlsx", sheet_name="Sheet1")

data = data.drop(columns=[
    "School Year",
    "School Term",
    "Campus Code",
    "Birth Place",
    "Birth City",
    "Place of Birth (Province)",
    "Birth Country"
])


# Example target variable (replace with your real one)
target = "Enrolled"

# Store results
results = []

for column in data.columns:
    if column != target and data[column].dtype == "object":  # Only categorical
        contingency_table = pd.crosstab(data[column], data[target])
        chi2, p, dof, expected = chi2_contingency(contingency_table)
        results.append((column, chi2, p))

significant_col = []

# Show results
for col, chi2, p in results:
    if p < 0.05:
        print(f"{col}: Chi2={chi2:.2f}, p={p:.4f} (Significant)")
        significant_col.append(col)
    else:
      print(f"{col}: Chi2={chi2:.2f}, p={p:.4f}")


print(significant_col)



ID: Chi2=20438.00, p=0.4928
Program (First Choice): Chi2=2057.94, p=0.0000 (Significant)
Program (Second Choice): Chi2=790.72, p=0.0000 (Significant)
Birth Date: Chi2=4683.45, p=0.0000 (Significant)
Gender: Chi2=92.79, p=0.0000 (Significant)
Citizen of: Chi2=13.34, p=0.3445
Religion: Chi2=115.96, p=0.0034 (Significant)
Civil Status: Chi2=19.11, p=0.0007 (Significant)
Current Region: Chi2=197.47, p=0.0000 (Significant)
Current Province: Chi2=368.28, p=0.0000 (Significant)
City/Municipality: Chi2=1060.02, p=0.0000 (Significant)
Current Brgy.: Chi2=4511.04, p=0.0001 (Significant)
Current Street: Chi2=11800.61, p=0.2483
Current Postal Code: Chi2=1322.12, p=0.0000 (Significant)
Complete Present Address: Chi2=15736.25, p=0.2583
Mobile Number: Chi2=16702.57, p=0.0767
Email: Chi2=20252.06, p=0.4787
Permanent Country: Chi2=10.11, p=0.2571
Permanent Region: Chi2=189.71, p=0.0000 (Significant)
Permanent Province: Chi2=548.05, p=0.0000 (Significant)
Permanent City: Chi2=1250.72, p=0.0000 (Signific

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency
import numpy as np

target = "Enrolled"
results = []

for col in data.select_dtypes(include="object").columns:
    if col != target:
        contingency_table = pd.crosstab(data[col], data[target])
        chi2, p, dof, expected = chi2_contingency(contingency_table)
        n = contingency_table.sum().sum()
        k = min(contingency_table.shape)  # min(rows, cols)
        cramer_v = np.sqrt((chi2 / n) / (k - 1)) if k > 1 else 0

        results.append((col, chi2, p, cramer_v))

# Put into DataFrame for ranking
results_df = pd.DataFrame(results, columns=["Column", "Chi2", "P-Value", "CramerV"])

# Keep only significant results
significant = results_df[results_df["P-Value"] < 0.05].sort_values("CramerV", ascending=False)

print(significant)


                     Column         Chi2       P-Value   CramerV
11            Current Brgy.  4511.042333  6.450817e-05  0.525266
21          Permanent Brgy.  4479.436743  1.544567e-04  0.523455
3                Birth Date  4683.448846  5.004382e-90  0.478700
26     Last School Attended  3077.641511  2.667340e-40  0.457734
1    Program (First Choice)  2057.935393  0.000000e+00  0.317319
23    Permanent Postal Code  1327.089420  1.470171e-20  0.288314
13      Current Postal Code  1322.124232  7.736650e-22  0.287639
20           Permanent City  1250.722368  9.973799e-24  0.276555
10        City/Municipality  1060.023664  1.361251e-32  0.254593
2   Program (Second Choice)   790.717898  4.802751e-88  0.251059
19       Permanent Province   548.054132  5.047916e-24  0.183074
9          Current Province   368.280534  2.430829e-41  0.150064
8            Current Region   197.465208  9.250465e-33  0.112605
18         Permanent Region   189.705090  3.327312e-31  0.110555
6                  Religi

In [ ]:
# Check potential spelling errors in all text columns
for col in data.select_dtypes(include="object").columns:
    print(f"\n=== {col} ===")
    counts = data[col].astype(str).value_counts().sort_index()
    print(counts)


=== ID ===
ID
100      1
1000     1
10000    1
10001    1
10002    1
        ..
9996     1
9997     1
9998     1
9999     1
id       3
Name: count, Length: 20436, dtype: int64

=== Program (First Choice) ===
Program (First Choice)
bsa-mla                   1213
bsaccountancy              604
bsarch                     500
bsarch-mla                 513
bsba-finmgt                793
bsba-mktgmgt               846
bsce                      3098
bscs                       203
bscs-mla                   443
bsit-mwa                  1172
bsmt                       918
bsmt-mla                  1760
bsn                       3657
bsn-mla                    985
bspsy                      750
bspsy-mla                 1808
bstm                       628
bstm-mla                   450
mm                          94
program (first choice)       3
Name: count, dtype: int64

=== Program (Second Choice) ===
Program (Second Choice)
air transportation major in advance flying      85
architecture  

In [ ]:
import pandas as pd
import numpy as np
import re

# ---------- Helpers ----------
def to_null_placeholders(df, extra_placeholders=None):
    # Base placeholders (lowercased for matching)
    base = {
        "none", "nan", "null", "n/a", "na",
        "gender", "civil status", "birth country", "citizen of",
        "program (first choice)", "program (second choice)",
        "school type", "student type",
        "current region", "permanent region", "permanent country",
        "--- select city ---", "-- select city --",
        "-- select from the list --", "- municipality -",
    }
    if extra_placeholders:
        base |= {x.strip().lower() for x in extra_placeholders}

    # Regex that flags strings made only of symbols like -, ., *, ?
    only_symbols_re = re.compile(r"^[\-\.\*\?]+$")

    for col in df.select_dtypes(include="object").columns:
        s = df[col]

        # Normalize for matching (but keep original series for assignment)
        norm = (
            s.astype(str)
             .str.strip()
             .str.lower()
        )

        mask_exact = norm.isin(base)

        # Also null out generic lone symbols / symbol-only strings
        mask_symbols = norm.str.match(only_symbols_re, na=False)

        # Also handle literal "nan" that slipped through
        mask_literal_nan = norm.eq("nan")

        # Collapse matches to None (NULL)
        df.loc[mask_exact | mask_symbols | mask_literal_nan, col] = None

    return df


def clean_ph_postal(series):
    """Keep only 4-digit PH postal codes; else set NULL. Also trims leading junk."""
    s = series.astype(str).str.strip()

    # remove any leading non-digits (e.g., '\t4217' -> '4217', '$021' -> '021')
    s = s.str.replace(r"^[^\d]+", "", regex=True)

    # keep only pure 4 digits; else NULL
    s = s.where(s.str.fullmatch(r"\d{4}"), None)

    return s


# ---------- Apply cleaning ----------
# 1) Null-out placeholders across all text columns
filtered_data = to_null_placeholders(data)

# 2) Postal code cleanup (kills random strings like 'yQKJD1yf')
for pc_col in ["Current Postal Code", "Permanent Postal Code"]:
    if pc_col in filtered_data.columns:
        filtered_data[pc_col] = clean_ph_postal(filtered_data[pc_col])

# (Optional) simple report of how many NULLs per cleaned column
report_cols = [c for c in filtered_data.columns if filtered_data[c].dtype == "object"]
null_report = filtered_data[report_cols].isna().sum().sort_values(ascending=False)
print("\nNULLs after placeholder cleanup (object columns):\n", null_report.head(20))

# Save
filtered_data.to_excel("Filtered_NoPlaceholders.xlsx", index=False)
print("✅ Placeholders removed → NULL; postal codes sanitized and saved to Filtered_NoPlaceholders.xlsx")



NULLs after placeholder cleanup (object columns):
 Program (Second Choice)       7896
School Type                   6471
Last School Attended          5750
Permanent Region              4920
Current Region                4868
Permanent Postal Code         4599
Current Postal Code           4569
Permanent Street              4504
Current Street                4492
Permanent Brgy.               4090
Current Brgy.                 4088
Permanent Country             4087
Permanent Province            4086
Permanent City                4086
City/Municipality             4084
Complete Present Address      4084
Complete Permanent Address    4084
Current Province              4084
Student Type                  3703
Religion                      3269
dtype: int64
✅ Placeholders removed → NULL; postal codes sanitized and saved to Filtered_NoPlaceholders.xlsx


In [ ]:
import pandas as pd
import re


# --- 2. Normalize religion ---
data["Religion"] = (
    data["Religion"]
    .replace({
        "roman catholic": "catholic",
        "catholic": "catholic",
    })
)

# --- 3. Clean 'Last School Attended' ---
def clean_school_name(value):
    if pd.isna(value):
        return None
    val = str(value).lower()
    # Patterns that indicate it's an address, not a school name
    if re.search(r"\d|street|st\.|purok|zone|#|,|/", val):
        return None
    return value

data["Last School Attended"] = data["Last School Attended"].apply(clean_school_name)

# --- 4. Clean Province columns ---
def clean_province(value):
    if pd.isna(value):
        return None
    val = str(value).strip().lower()
    if val.isnumeric() or val in ["4a"]:  # drop numeric codes
        return None
    if "4a calabarzon" in val:
        return "calabarzon"
    return val

for col in ["Current Province", "Permanent Province"]:
    data[col] = data[col].apply(clean_province)

# --- 5. Clean City/Municipality columns ---
def clean_city(value):
    if pd.isna(value):
        return None
    val = str(value).strip().lower()
    if val.isnumeric():  # drop numeric-only cities
        return None
    return val

for col in ["City/Municipality", "Permanent City"]:
    data[col] = data[col].apply(clean_city)

# Save updated dataset
data.to_excel("Filtered_DataSet_Cleaned.xlsx", index=False)

print("Cleaning done! Saved as Filtered_DataSet_Cleaned.xlsx")



Cleaning done! Saved as Filtered_DataSet_Cleaned.xlsx


In [ ]:
pip install rapidfuzz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 27.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from rapidfuzz import fuzz, process

# Load dataset
file_path = "/content/Filtered_DataSet_Cleaned.xlsx"
data = pd.read_excel(file_path)

# Only check object (text) columns
text_columns = data.select_dtypes(include="object").columns

# Function to find potential spelling issues
def find_similar_values(series, threshold=85):
    unique_vals = series.dropna().unique()
    checked = set()
    groups = []

    for val in unique_vals:
        if val in checked:
            continue
        matches = process.extract(val, unique_vals, scorer=fuzz.ratio)
        # keep only those with similarity >= threshold
        close = [m[0] for m in matches if m[1] >= threshold and m[0] != val]
        if close:
            groups.append((val, close))
            checked.update(close)
            checked.add(val)
    return groups

# Check each column
for col in text_columns:
    print(f"\n=== Checking '{col}' for possible spelling errors ===")
    groups = find_similar_values(data[col].astype(str), threshold=85)
    if not groups:
        print("No close matches found.")
    else:
        for base, similars in groups:
            print(f"'{base}' may be confused with: {similars}")


KeyboardInterrupt: 

Exception ignored in: 'zmq.backend.cython._zmq.Frame.__del__'
Traceback (most recent call last):
  File "_zmq.py", line 160, in zmq.backend.cython._zmq._check_rc
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
file_path = "/content/Filtered_DataSet_Cleaned.xlsx"
data = pd.read_excel(file_path, sheet_name="Sheet1")

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency
import numpy as np

target = "Enrolled"
results = []

for col in data.select_dtypes(include="object").columns:
    if col != target:
        contingency_table = pd.crosstab(data[col], data[target])
        chi2, p, dof, expected = chi2_contingency(contingency_table)
        n = contingency_table.sum().sum()
        k = min(contingency_table.shape)  # min(rows, cols)
        cramer_v = np.sqrt((chi2 / n) / (k - 1)) if k > 1 else 0

        results.append((col, chi2, p, cramer_v))

# Put into DataFrame for ranking
results_df = pd.DataFrame(results, columns=["Column", "Chi2", "P-Value", "CramerV"])

# Keep only significant results
significant = results_df[results_df["P-Value"] < 0.05].sort_values("CramerV", ascending=False)

print(significant)


                     Column         Chi2       P-Value   CramerV
11            Current Brgy.  4511.042333  6.450817e-05  0.525266
20          Permanent Brgy.  4479.436743  1.544567e-04  0.523455
3                Birth Date  4683.448846  5.004382e-90  0.478700
24     Last School Attended  2696.227764  1.528330e-35  0.456486
1    Program (First Choice)  2053.311363  0.000000e+00  0.316986
19           Permanent City  1248.716647  9.089511e-24  0.276350
10        City/Municipality  1060.023664  1.361251e-32  0.254593
2   Program (Second Choice)   788.766393  4.590599e-88  0.250779
18       Permanent Province   545.872203  2.801803e-24  0.182720
9          Current Province   368.280534  2.430829e-41  0.150064
8            Current Region   194.275760  1.134908e-32  0.111703
17         Permanent Region   186.525457  4.125953e-31  0.109635
6                  Religion   112.486241  5.200919e-03  0.080943
4                    Gender    88.913911  4.123732e-21  0.071964
25              School Ty

In [ ]:
print(data.columns)

Index(['ID', 'Program (First Choice)', 'Program (Second Choice)', 'Birth Date',
       'Gender', 'Citizen of', 'Religion', 'Civil Status', 'Current Region',
       'Current Province', 'City/Municipality', 'Current Brgy.',
       'Current Street', 'Current Postal Code', 'Complete Present Address',
       'Mobile Number', 'Email', 'Permanent Country', 'Permanent Region',
       'Permanent Province', 'Permanent City', 'Permanent Brgy.',
       'Permanent Street', 'Permanent Postal Code',
       'Complete Permanent Address', 'Student Type', 'Last School Attended',
       'School Type', 'Enrolled'],
      dtype='object')


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load your data
file_path = "/content/Filtered_DataSet_Cleaned.xlsx"
data = pd.read_excel(file_path, sheet_name="Sheet1")
data = data.drop(columns=[
    "ID",
    "Civil Status",
    "Gender",
    "Religion",
    "Mobile Number",
    "Email",
    "Current Postal Code",
    "Permanent Postal Code",
    "Complete Present Address",
    "Complete Permanent Address",
    "Citizen of"
])


In [ ]:
program_map = {
    "bsn":"bachelor of science in nursing",
    "bsce":"bachelor of science in civil engineering",
    "bspsy-mla":"bachelor of science in psychology",
    "bsmt-mla":"bachelor of science in medical technology",
    "bsa-mla": "bachelor of science in accountancy",
    "bsit-mwa": "bachelor of science in information technology",
    "bsn-mla": "bachelor of science in nursing",
    "bsmt": "bachelor of science in medical technology",
    "bsba-mktgmgt": "bachelor of science in business administration major in marketing management",
    "bsba-finmgt": "bachelor of science in business administration major in financial management",
    "bspsy": "bachelor of science in psychology",
    "bstm" : "bachelor of science in tourism management",
    "bsaccountancy": "bachelor of science in accountancy",
    "bsarch-mla": "bachelor of science in architecture",
    "bsarch": "bachelor of science in architecture",
    "bstm-mla": "bachelor of science in tourism management",
    "bscs-mla": "bachelor of science in computer science",
    "bscs": "bachelor of science in computer science",
    "mm": "masters of management"
}

data['Program (First Choice)'] = data['Program (First Choice)'].map(program_map)

In [ ]:
import pandas as pd
from datetime import datetime

# ✅ Step 1: Convert to datetime safely
data["Birth Date"] = pd.to_datetime(
    data["Birth Date"], errors="coerce", dayfirst=True
)

# ✅ Step 2: Extract year
data["BirthYear"] = data["Birth Date"].dt.year

# ✅ Step 3: Filter unrealistic years
current_year = datetime.now().year
valid_mask = (data["BirthYear"] >= 1900) & (data["BirthYear"] <= current_year)
data.loc[~valid_mask, "BirthYear"] = None

# ✅ Step 4: Compute Age
data["Age"] = current_year - data["BirthYear"]

data = data.drop(columns=["Birth Date", "BirthYear"])


# Save updated dataset
data.to_excel("cleaned.xlsx", index=False)

/tmp/ipython-input-3760701857.py:5: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  data["Birth Date"] = pd.to_datetime(
